<a href="https://colab.research.google.com/github/balrajparsi/balrajparsi/blob/AI_assigenments/Balraj_Parsi_lstm_assignment4_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🥙 LSTM on Recipe Data

In this notebook, we'll walk through the steps required to train your own LSTM on the recipes dataset

In [1]:
# (Optional) Autoreload is disabled for compatibility in some environments (e.g., Python 3.12).
# If you are on a supported IPython environment and want it, uncomment the next two lines.
# %load_ext autoreload
# %autoreload 2


In [2]:
import os
import json
import re
import string
import random
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks, losses
import pandas as pd
from tqdm import tqdm

In [3]:
import tensorflow as tf

# Check GPU availability
print('TensorFlow:', tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
print('GPUs:', gpus)

# (Recommended) prevent TF from pre-allocating all GPU memory
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print('Memory growth enabled')
    except Exception as e:
        print('Could not set memory growth:', e)

# Optional: mixed precision can speed up training on modern GPUs (T4/V100/A100)
USE_MIXED_PRECISION = bool(gpus)
if USE_MIXED_PRECISION:
    from tensorflow.keras import mixed_precision
    mixed_precision.set_global_policy('mixed_float16')
    print('Mixed precision enabled: mixed_float16')
else:
    print('Mixed precision disabled (no GPU detected)')

AUTOTUNE = tf.data.AUTOTUNE


TensorFlow: 2.19.0
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Memory growth enabled
Mixed precision enabled: mixed_float16


## 0. Parameters <a name="parameters"></a>

In [4]:
VOCAB_SIZE = 10000
MAX_LEN = 200
EMBEDDING_DIM = 100
N_UNITS = 128
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False

# If a GPU is available, a larger batch size is usually faster.
# You can tune this (64/128/256) depending on GPU memory.
BATCH_SIZE = 128 if len(tf.config.list_physical_devices("GPU")) > 0 else 32

EPOCHS = 25


## 1. Load the data <a name="load"></a>

In [5]:
# Load the full dataset
with open("/content/full_format_recipes.json") as json_data:
    recipe_data = json.load(json_data)

In [6]:
# Filter the dataset
filtered_data = [
    "Recipe for " + x["title"] + " | " + " ".join(x["directions"])
    for x in recipe_data
    if "title" in x
    and x["title"] is not None
    and "directions" in x
    and x["directions"] is not None
]

In [7]:
# Count the recipes
n_recipes = len(filtered_data)
print(f"{n_recipes} recipes loaded")

20111 recipes loaded


In [8]:
example = filtered_data[9]
print(example)

Recipe for Ham Persillade with Mustard Potato Salad and Mashed Peas  | Chop enough parsley leaves to measure 1 tablespoon; reserve. Chop remaining leaves and stems and simmer with broth and garlic in a small saucepan, covered, 5 minutes. Meanwhile, sprinkle gelatin over water in a medium bowl and let soften 1 minute. Strain broth through a fine-mesh sieve into bowl with gelatin and stir to dissolve. Season with salt and pepper. Set bowl in an ice bath and cool to room temperature, stirring. Toss ham with reserved parsley and divide among jars. Pour gelatin on top and chill until set, at least 1 hour. Whisk together mayonnaise, mustard, vinegar, 1/4 teaspoon salt, and 1/4 teaspoon pepper in a large bowl. Stir in celery, cornichons, and potatoes. Pulse peas with marjoram, oil, 1/2 teaspoon pepper, and 1/4 teaspoon salt in a food processor to a coarse mash. Layer peas, then potato salad, over ham.


## 2. Tokenise the data

In [9]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


text_data = [pad_punctuation(x) for x in filtered_data]

In [10]:
# Display an example of a recipe
example_data = text_data[9]
example_data

'Recipe for Ham Persillade with Mustard Potato Salad and Mashed Peas | Chop enough parsley leaves to measure 1 tablespoon ; reserve . Chop remaining leaves and stems and simmer with broth and garlic in a small saucepan , covered , 5 minutes . Meanwhile , sprinkle gelatin over water in a medium bowl and let soften 1 minute . Strain broth through a fine - mesh sieve into bowl with gelatin and stir to dissolve . Season with salt and pepper . Set bowl in an ice bath and cool to room temperature , stirring . Toss ham with reserved parsley and divide among jars . Pour gelatin on top and chill until set , at least 1 hour . Whisk together mayonnaise , mustard , vinegar , 1 / 4 teaspoon salt , and 1 / 4 teaspoon pepper in a large bowl . Stir in celery , cornichons , and potatoes . Pulse peas with marjoram , oil , 1 / 2 teaspoon pepper , and 1 / 4 teaspoon salt in a food processor to a coarse mash . Layer peas , then potato salad , over ham . '

In [11]:
# Convert to a TensorFlow Dataset (optimized pipeline)
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .shuffle(10000, seed=SEED, reshuffle_each_iteration=True)
    .batch(BATCH_SIZE, drop_remainder=True)
    # .cache()  # uncomment if you have enough RAM (can speed up epochs)
    .prefetch(AUTOTUNE)
)


In [12]:
# Create a vectorisation layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [13]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [14]:
# Display some token:word mappings
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: .
3: ,
4: and
5: to
6: in
7: the
8: with
9: a


In [15]:
# Display the same example converted to ints
example_tokenised = vectorize_layer(example_data)
print(example_tokenised.numpy())

[  26   16  557    1    8  298  335  189    4 1056  498   27  332  228
  235  262    5  594   11  134   22  311    2  332   45  262    4  671
    4   70    8  171    4   81    6    9   65   80    3  121    3   59
   12    2  299    3   88  650   20   39    6    9   29   21    4   67
  529   11  164    2  320  171  102    9  374   13  642  307   25   21
    8  650    4   42    5  931    2   63    8   24    4   33    2  114
   21    6  178  181 1244    4   60    5  140  112    3   48    2  117
  557    8  285  235    4  200  292  980    2  107  650   28   72    4
  108   10  114    3   57  204   11  172    2   73  110  482    3  298
    3  190    3   11   23   32  143   24    3    4   11   23   32  143
   33    6    9   30   21    2   42    6  353    3 3223    3    4  150
    2  437  498    8 1281    3   37    3   11   23   15  143   33    3
    4   11   23   32  143   24    6    9  291  188    5    9  412  573
    2  230  498    3   46  335  189    3   20  557    2    0    0    0
    0 

## 3. Create the Training Set

In [16]:
# Create the training set of recipes and the same text shifted by one word
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

## 4. Build the LSTM <a name="build"></a>

In [17]:
inputs = layers.Input(shape=(None,), dtype="int32")
x = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x = layers.LSTM(N_UNITS, return_sequences=True)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
lstm = models.Model(inputs, outputs)
lstm.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 100)      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 128)      │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 10000)    │     1,290,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,407,248 (9.18 MB)

 Trainable params: 2,407,248 (9.18 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
if LOAD_MODEL:
    # model.load_weights('./models/model')
    lstm = models.load_model("./models/lstm", compile=False)

## 5. Train the LSTM <a name="train"></a>

In [19]:
loss_fn = losses.SparseCategoricalCrossentropy()

# jit_compile can speed up training on GPU (XLA). Safe to disable if you hit issues.
USE_XLA = len(tf.config.list_physical_devices('GPU')) > 0
lstm.compile('adam', loss_fn, jit_compile=USE_XLA)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/trainer.py:212: UserWarning: Model doesn't support `jit_compile=True`. Proceeding with `jit_compile=False`.
  warnings.warn(


In [20]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }  # <1>

    def sample_from(self, probs, temperature):  # <2>
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]  # <3>
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:  # <4>
            x = np.array([start_tokens])
            y = self.model.predict(x, verbose=0)  # <5>
            sample_token, probs = self.sample_from(y[0][-1], temperature)  # <6>
            info.append({"prompt": start_prompt, "word_probs": probs})
            start_tokens.append(sample_token)  # <7>
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("recipe for", max_tokens=100, temperature=1.0)

In [21]:
# Create a model save checkpoint
import os
from tensorflow.keras import callbacks

os.makedirs("./checkpoint", exist_ok=True)

model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.weights.h5",  # must end with .weights.h5
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")


# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [22]:
lstm.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step - loss: 6.3235
generated text:
recipe for 30 mesh nights little divide heat cook pour juice serving saucepan and 4 salted 

157/157 ━━━━━━━━━━━━━━━━━━━━ 54s 309ms/step - loss: 6.3162
Epoch 2/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step - loss: 4.1807
generated text:
recipe for 9x6 are traction unfurling . 

157/157 ━━━━━━━━━━━━━━━━━━━━ 46s 294ms/step - loss: 4.1799
Epoch 3/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step - loss: 3.6747
generated text:
recipe for arancine allowing grilled oven | until simmering - cardamom paste bits ; stirring sauce and chill to on a oven thermometer , and 4 cinnamon with sprinkle fish to 350°f . cut sliding sugar to 350°f , - skewer of . sauté 1 / in pepper . set finger , 5 minutes until small pinkness is bleed crease . let brisket basic half , with with salt , and 4 ; blender . bring mixture of in processor , 1 / until meat and sprinkle , cook heat 1 tablespoon ingredients , undisturbed to 1 / / 4

1

In [24]:
# Save the final model
os.makedirs("./models", exist_ok=True)

lstm.save("./models/lstm.keras")

## 6. Generate text using the LSTM

In [25]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        print(f"\nPROMPT: {i['prompt']}")
        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [26]:
info = text_generator.generate(
    "recipe for roasted vegetables | chop 1 /", max_tokens=80, temperature=1.4
)


generated text:
recipe for roasted vegetables | chop 1 / 2 cup yogurt , reserving sides for you ; tilt , pressing cereal into oz by disturbing . 4 empty skillets . bring to boil ; force over ham vinegar mixture . let salami ; coat salad into amazon . beat dressing in bits and smooth rub in crème verts . drizzle braids with 4 crème fraîche tartare - cut frozen bacon aged marjoram on plates ; 



In [27]:
print_probs(info, vocab)


PROMPT: recipe for roasted vegetables | chop 1 /
2:   	35.09375%
4:   	29.875%
3:   	9.6171875%
8:   	5.94921875%
1:   	1.23046875%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 2
cup:   	26.375%
teaspoon:   	12.9765625%
-:   	5.26171875%
tablespoon:   	4.28125%
tsp:   	3.91015625%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 2 cup
oil:   	2.900390625%
water:   	1.8701171875%
chopped:   	1.2900390625%
garlic:   	1.2197265625%
green:   	0.97998046875%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 2 cup yogurt
in:   	24.953125%
,:   	13.0703125%
and:   	11.703125%
.:   	5.8515625%
into:   	3.169921875%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 2 cup yogurt ,
then:   	9.2890625%
reserving:   	3.619140625%
and:   	2.890625%
in:   	2.08984375%
cut:   	1.650390625%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 2 cup yogurt , reserving
1:   	9.7109375%
2:   	3.16015625%
the:   	2.529296875%
3:   	1.6396484375%
remaind

In [28]:
info = text_generator.generate(
    "recipe for roasted vegetables | chop 1 /", max_tokens=80, temperature=0.6
)


generated text:
recipe for roasted vegetables | chop 1 / 4 cup fresh water and in a small saucepan , add the onion , and beans with water to cover by 2 inches , and bring to a boil . cook , stirring , until the liquid is absorbed . add the milk and cook , stirring , for 5 minutes . add the onion and cook , stirring , until it is golden . stir in the soy sauce , the



In [29]:
print_probs(info, vocab)


PROMPT: recipe for roasted vegetables | chop 1 /
2:   	57.125%
4:   	39.15625%
3:   	2.779296875%
8:   	0.91015625%
1:   	0.0200042724609375%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 4
cup:   	90.9375%
teaspoon:   	7.26953125%
-:   	1.0703125%
tsp:   	0.340087890625%
tablespoon:   	0.1199951171875%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 4 cup
oil:   	39.34375%
water:   	9.640625%
chopped:   	3.76953125%
garlic:   	3.580078125%
green:   	2.560546875%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 4 cup fresh
garlic:   	20.03125%
onion:   	10.3828125%
green:   	7.44140625%
lemon:   	5.1015625%
water:   	4.6015625%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 4 cup fresh water
and:   	35.09375%
in:   	29.21875%
,:   	26.34375%
.:   	8.6015625%
;:   	0.31005859375%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 4 cup fresh water and
in:   	12.3984375%
1:   	10.671875%
cut:   	10.0%
place:   	6.01953125%
pat: 

In [30]:
info = text_generator.generate(
    "recipe for chocolate ice cream |", max_tokens=60, temperature=1.4
)
print_probs(info, vocab)


generated text:
recipe for chocolate ice cream | beat cream gnocchi over brisk inside and beat turns . beat cookie carton , pressing firmly as against speed , resulting granny so—and in an ice patties in beer - diameter forward flutes of atop [UNK] ; fill fountain cake dishes . cakes may be made up strips wrapped in covered , by a


PROMPT: recipe for chocolate ice cream |
combine:   	7.078125%
in:   	7.0%
preheat:   	3.529296875%
blend:   	3.150390625%
bring:   	3.099609375%
--------


PROMPT: recipe for chocolate ice cream | beat
cream:   	10.34375%
egg:   	5.78125%
sugar:   	5.109375%
1:   	3.48046875%
eggs:   	2.80078125%
--------


PROMPT: recipe for chocolate ice cream | beat cream
in:   	9.46875%
and:   	8.796875%
cheese:   	7.8203125%
with:   	7.5703125%
,:   	7.12109375%
--------


PROMPT: recipe for chocolate ice cream | beat cream gnocchi
in:   	17.3125%
,:   	11.90625%
on:   	10.25%
with:   	8.0234375%
until:   	4.1484375%
--------


PROMPT: recipe for chocolate ice cream 

In [31]:
info = text_generator.generate(
    "recipe for chocolate ice cream |", max_tokens=60, temperature=0.6
)
print_probs(info, vocab)


generated text:
recipe for chocolate ice cream | blend flour and milk in small bowl . let stand , covered , until dough comes out and cool . ( can be made 2 days ahead . keep refrigerated . ) 


PROMPT: recipe for chocolate ice cream |
combine:   	33.1875%
in:   	32.375%
preheat:   	6.578125%
blend:   	5.03125%
bring:   	4.87109375%
--------


PROMPT: recipe for chocolate ice cream | blend
all:   	29.9375%
flour:   	24.640625%
first:   	13.90625%
together:   	11.4296875%
1:   	3.759765625%
--------


PROMPT: recipe for chocolate ice cream | blend flour
,:   	95.25%
and:   	4.19140625%
in:   	0.0999755859375%
together:   	0.0999755859375%
ingredients:   	0.08001708984375%
--------


PROMPT: recipe for chocolate ice cream | blend flour and
sugar:   	42.59375%
1:   	16.453125%
baking:   	15.8203125%
flour:   	4.76953125%
salt:   	3.0703125%
--------


PROMPT: recipe for chocolate ice cream | blend flour and milk
in:   	99.8125%
with:   	0.029998779296875%
,:   	0.029998779296875%
on:   	